In [1]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd

from os.path import basename

In [2]:
# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)
            
nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

In [3]:
# .json file loop
#df = pd.DataFrame(columns=['Function','Device','Data_Type',
#                           'Num_Sc','Range_Sc','Num_ScI','Range_ScI',
#                           'Num_Arr','Range_Arr'])

listD = ["tsim","simulavr","Thumbulator","armulator",] # Inserisci i dispositivi che vuoi analizzare
listDT = ["int8_t"] # Inserisci i data type che vuoi analizzare

listrange = [0, 0]
listrangeI = [0, 0]
listdtI = [' ']
listdt = [' ']
listfunI = [' ']
listfun = [' ']
listdevI = [' ']
listdev = [' ']
listboardI = [' ']
listboard = [' ']

for i in range(0, len(pathFile)):
    pF = pathFile[i]

    with open(pF, 'r') as jf:
        data = jf.read()
    obj = json.loads(data)

    if os.name == 'nt':
        spR = rootFile[i].split('\\')
    else:
        spR = rootFile[i].split('/')
    function = spR[-1]
    board = spR[-3]
    
    #Device loop
    deviceList = list(obj.keys())
    deviceListR = []
    for u in range(0, len(deviceList)):
        if deviceList[u] in listD:
            deviceListR.append(deviceList[u])

    for n in range(0, len(deviceListR)):
        # Data_type loop
        device = deviceListR[n]
        dtarr = list(obj[deviceListR[n]].keys())
        
        dtarrR = []
        for u in range(0, len(dtarr)):
            if dtarr[u] in listDT:
                dtarrR.append(dtarr[u])
        
        for j in range(0, len(dtarrR)):
            dataype = dtarrR[j]
            lsubNP = (list(obj[deviceListR[n]][dtarrR[j]].keys()))
            lsubVP = (list(obj[deviceListR[n]][dtarrR[j]].values()))
            ltot = len(lsubNP)

            kl = []
            for k in range(0, ltot):
                flag = 0
                if not "[" in lsubNP[k]:
                    for keysl in lsubNP:
                        if (lsubNP[k] in keysl) and (not lsubNP[k] is keysl) and (flag == 0):          
                            flag = 1
                            kl.append(k) 

            m = 0
            for item in lsubVP:
                #print(function, lsubVP)
                if ';' in item:
                    newstr = item.split(";")[0]
                    newstr = newstr.replace("[", "")
                    newstr = newstr.replace("]", "")
                    el1 = float(newstr.split(",")[0])
                    el2 = float(newstr.split(",")[1])
                    strint = el1
                    rangeint = el2 - el1
                    lr = np.array([strint, rangeint])
                    if m in kl:
                        listrangeI = np.vstack([listrangeI, lr])
                        listdtI.append(dataype)
                        listfunI.append(function)
                        listboardI.append(board)
                        listdevI.append(device)
                    else:
                        listrange = np.vstack([listrange, lr])
                        listdt.append(dataype)
                        listfun.append(function)
                        listboard.append(board)
                        listdev.append(device)
                m = m + 1

In [4]:
listrangeS = pd.DataFrame(listrange)
listdtS = pd.Series(listdt)
listfunS = pd.Series(listfun)
listdevS = pd.Series(listdev)
listboardS = pd.Series(listboard)
dfS = pd.concat([listdevS, listboardS, listfunS, listdtS, listrangeS], axis=1)
dfS.columns = ['DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'DATA_START', 'DATA_RANGE']
dfS = dfS.iloc[1:]

In [5]:
dfS

,DEVICE,BOARD,FUNCTION,DATA_TYPE,DATA_START,DATA_RANGE
1,armulator,ARMV4T,binary_search,int8_t,2.0,26.0
2,armulator,ARMV4T,fibcall,int8_t,2.0,9.0
3,armulator,ARMV4T,gcd,int8_t,2.0,125.0
4,armulator,ARMV4T,gcd,int8_t,2.0,125.0
5,armulator,ARMV4T,park_miller,int8_t,2.0,9.0
6,simulavr,Atmega328p,binary_search,int8_t,2.0,26.0
7,simulavr,Atmega328p,fibcall,int8_t,2.0,9.0
8,simulavr,Atmega328p,gcd,int8_t,2.0,125.0
9,simulavr,Atmega328p,gcd,int8_t,2.0,125.0
10,simulavr,Atmega328p,park_miller,int8_t,2.0,9.0


In [6]:
listrangeIS = pd.DataFrame(listrangeI)
listdtIS = pd.Series(listdtI)
listfunIS = pd.Series(listfunI)
listdevIS = pd.Series(listdevI)
listboardIS = pd.Series(listboardI)
dfSI = pd.concat([listdevIS, listboardIS , listfunIS, listdtIS, listrangeIS], axis=1)
dfSI.columns = ['DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'DATA_START', 'DATA_RANGE']
dfSI = dfSI.iloc[1:]

In [7]:
dfSI

,DEVICE,BOARD,FUNCTION,DATA_TYPE,DATA_START,DATA_RANGE
1,armulator,ARMV4T,banker_algorithm,int8_t,2.0,26.0
2,armulator,ARMV4T,banker_algorithm,int8_t,2.0,26.0
3,armulator,ARMV4T,bellmanford,int8_t,2.0,26.0
4,armulator,ARMV4T,bfs,int8_t,2.0,26.0
5,armulator,ARMV4T,binary_search,int8_t,2.0,26.0
6,armulator,ARMV4T,bubble_sort,int8_t,2.0,26.0
7,armulator,ARMV4T,insertionsort,int8_t,2.0,26.0
8,armulator,ARMV4T,kruskal,int8_t,2.0,26.0
9,armulator,ARMV4T,matrix_mult,int8_t,2.0,13.0
10,armulator,ARMV4T,matrix_mult,int8_t,2.0,13.0


In [8]:
serS = dfS.groupby(['DATA_TYPE'])['DATA_RANGE'].max()

n = 3
ws = 10
arrlM = []
for d in range(0, len(serS)):
    arrlimt = []
    ml = 0
    w = (serS.iloc[d] - ml) // n
    for h in range(n):
        arrlimt.append(ml + h*w)
    arrlM.append(arrlimt)
    
for index, row in dfS.iterrows():
    clf = 0
    for p in range(0, len(serS)):
        if row['DATA_TYPE'] == serS.index[p]:
            ind = p
            arrl = arrlM[ind]
            for u in range(0, len(arrl)-1):
                if (row['DATA_RANGE'] > arrl[u]) and (row['DATA_RANGE'] <= arrl[u+1]):
                    clf = u + 1
                elif (row['DATA_RANGE'] > arrl[u+1]):
                    clf = len(arrl)            
    dfS['RANGE_S'] = clf 

In [9]:
serSI = dfSI.groupby(['DATA_TYPE'])['DATA_RANGE'].max()

n = 3
ws = 10
arrlM = []
for d in range(0, len(serSI)):
    arrlimt = []
    ml = 0
    w = (serSI.iloc[d] - ml) // n
    for h in range(n):
        arrlimt.append(ml + h*w)
    arrlM.append(arrlimt)
    
for index, row in dfSI.iterrows():
    clf = 0
    for p in range(0, len(serSI)):
        if row['DATA_TYPE'] == serSI.index[p]:
            ind = p
            arrl = arrlM[ind]
            for u in range(0, len(arrl)-1):
                if (row['DATA_RANGE'] > arrl[u]) and (row['DATA_RANGE'] <= arrl[u+1]):
                    clf = u + 1
                elif (row['DATA_RANGE'] > arrl[u+1]):
                    clf = len(arrl)            
    dfSI['RANGE_SI'] = clf 

In [10]:
dfS.to_csv("dfS_tot.csv", sep=';')
dfSI.to_csv("dfSI_tot.csv", sep=';')

In [11]:
dfS.to_csv("dfS.csv", sep=';')
dfSI.to_csv("dfSI.csv", sep=';')